# Jobs & Results

Quantinuum Nexus offers different types of `Job` that represent a component of your workflow that is running in Nexus, H-series or a third-party.

- CompileJobs represent the TKET compilation of circuits for a particular target device.
- ExecuteJobs represent the execution of circuits on a quantum computer or simulator.

Nexus manages the storage of Job outputs such as BackendResults, BackendInfo or compiled Circuits.

In [1]:
from datetime import datetime

import qnexus as qnx

from pytket import Circuit

In [2]:
my_job_name_prefix = datetime.now()

my_project_ref = qnx.projects.get_or_create(name="My Project")

my_circuit_ref = qnx.circuits.upload(
    name=f"My Circuit from {datetime.now()}",
    circuit = Circuit(2).H(0).CX(0,1).measure_all(),
    project = my_project_ref,
)

## Compile Jobs

These jobs represent the TKET compilation of one or more circuit(s) in Nexus for a particular target backend.

BackendConfigs define the target, in this case we are targetting the `H1-1SC` H-series syntax checker device.

In [4]:
# Compile the circuit (blocking), to receive a list of compiled CircuitRefs

compiled_circuits = qnx.compile(
    circuits=[my_circuit_ref],
    name=f"{my_job_name_prefix}_compile",
    optimisation_level=1,
    backend_config=qnx.QuantinuumConfig(device_name="H1-1SC"),
    project=my_project_ref,
)

compiled_circuits.df()

,name,description,created,modified,project,id
0,My Circuit from 2024-07-18 18:32:34.164662-Qua...,None,2024-07-18 17:32:59.616512+00:00,2024-07-18 17:32:59.616512+00:00,My Nexus Project,f9168b6b-ca8e-4b68-b624-2c2fb336c7fd


In [6]:
# Run in an asyncronous manner to receieve a JobRef

compile_job_ref = qnx.start_compile_job(
    circuits=[my_circuit_ref],
    name=f"{my_job_name_prefix}_compile_async",
    optimisation_level=1,
    backend_config=qnx.QuantinuumConfig(device_name="H1-1SC"),
    project=my_project_ref
)

# Block until the job is complete (or perform other tasks while we wait)
qnx.jobs.wait_for(compile_job_ref)

# Retrieve a CompilationResultRef for every Circuit that was compiled
compile_job_result_refs = qnx.jobs.results(compile_job_ref)

compile_job_result_refs.df()


,name,description,created,modified,project,id
0,My Circuit from 2024-07-18 18:32:34.164662-com...,,2024-07-18 17:33:50.696154+00:00,2024-07-18 17:33:50.696154+00:00,My Nexus Project,7b109e7e-010c-4336-b9f8-e9d37c0f281c


In [7]:
# Retrieve the compiled CircuitRef for the first Circuit
compile_job_result_refs[0].get_output()

CircuitRef(id=UUID('984cdfd1-e78d-4041-aaf3-39131e877531'), annotations=Annotations(name='My Circuit from 2024-07-18 18:32:34.164662-QuantinuumBackend-final', description=None, properties=OrderedDict(), created=datetime.datetime(2024, 7, 18, 17, 33, 50, 504675, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 7, 18, 17, 33, 50, 504675, tzinfo=TzInfo(UTC))), project=ProjectRef(id=UUID('e1e53608-5e73-4b96-b15f-96e727f61e47'), annotations=Annotations(name='My Nexus Project', description=None, properties=OrderedDict(), created=datetime.datetime(2024, 7, 18, 17, 24, 2, 870970, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 7, 18, 17, 24, 2, 870970, tzinfo=TzInfo(UTC))), contents_modified=datetime.datetime(2024, 7, 18, 17, 33, 51, 593300, tzinfo=TzInfo(UTC)), type='ProjectRef'), type='CircuitRef')

In [8]:
# View the compilation passes that we applied when compiling the circuit
compile_job_result_refs[0].get_passes().df()

,pass name,input,output,id
0,DecomposeBoxes,My Circuit from 2024-07-18 18:32:34.164662,My Circuit from 2024-07-18 18:32:34.164662,668ac4bf-bca3-4ef4-a43d-3c8802fcce00
1,CustomPass,My Circuit from 2024-07-18 18:32:34.164662,My Circuit from 2024-07-18 18:32:34.164662,bdb7058d-8909-4403-be2e-5ce9050406f9
2,SynthesiseTK,My Circuit from 2024-07-18 18:32:34.164662,My Circuit from 2024-07-18 18:32:34.164662-Qua...,5b8f907a-2390-423f-8073-a108aa7ad38e
3,NormaliseTK2,My Circuit from 2024-07-18 18:32:34.164662-Qua...,My Circuit from 2024-07-18 18:32:34.164662-Qua...,45e069bf-467e-466c-824e-4f48fb51d1bd
4,DecomposeTK2,My Circuit from 2024-07-18 18:32:34.164662-Qua...,My Circuit from 2024-07-18 18:32:34.164662-Qua...,2b9cec89-7fe2-4339-b050-ae0d30b5b37d
5,AutoRebase,My Circuit from 2024-07-18 18:32:34.164662-Qua...,My Circuit from 2024-07-18 18:32:34.164662-Qua...,fde872fc-5120-4e18-a8f7-71ce00ca9695
6,ZZPhaseToRz,My Circuit from 2024-07-18 18:32:34.164662-Qua...,My Circuit from 2024-07-18 18:32:34.164662-Qua...,f16c4130-1de1-4e66-9506-6b8354d060ff
7,RemoveRedundancies,My Circuit from 2024-07-18 18:32:34.164662-Qua...,My Circuit from 2024-07-18 18:32:34.164662-Qua...,11d32f91-e88c-4279-a3d1-c14f4b124875
8,AutoSquash,My Circuit from 2024-07-18 18:32:34.164662-Qua...,My Circuit from 2024-07-18 18:32:34.164662-Qua...,e71d05dd-31cf-4b40-b5a6-fe885ec1058f
9,RemoveRedundancies,My Circuit from 2024-07-18 18:32:34.164662-Qua...,My Circuit from 2024-07-18 18:32:34.164662-Qua...,45f6a322-aad8-4d5d-a9d0-ae4f5897fb87


## Execute Jobs

These jobs represent the execution of one or more circuit(s) on a quantum computer or simulator.

In [13]:
# Execute the circuit (blocking), to receive a list of pytket BackendResults

results = qnx.execute(
    circuits=compiled_circuits,
    name=f"{my_job_name_prefix}_execute",
    n_shots=[100]* len(compiled_circuits),
    backend_config=qnx.QuantinuumConfig(device_name="H1-1SC"),
    project=my_project_ref,
)

results[0].get_counts()

Counter({(0, 0): 100})

In [14]:
# Run in an asyncronous manner to receieve a JobRef

execute_job_ref = qnx.start_execute_job(
    circuits=compiled_circuits,
    name=f"{my_job_name_prefix}_execute_async",
    n_shots=[100]* len(compiled_circuits),
    backend_config=qnx.QuantinuumConfig(device_name="H1-1SC"),
    project=my_project_ref,
)

# Block until the job is complete (or perform other tasks while we wait)
qnx.jobs.wait_for(execute_job_ref)

# Retrieve a ExecutionResultRef for every Circuit that was run
execute_job_result_refs = qnx.jobs.results(execute_job_ref)

execute_job_result_refs.df()

,name,description,created,modified,project,id
0,2024-07-18 18:35:03.915748_execute_async,,2024-07-18 17:35:35.971074+00:00,2024-07-18 17:35:35.971074+00:00,My Nexus Project,08686fa0-087f-465f-a785-7e2ee3864b37


In [15]:
# Get the input CircuitRef
execute_job_result_refs[0].get_input()

CircuitRef(id=UUID('f9168b6b-ca8e-4b68-b624-2c2fb336c7fd'), annotations=Annotations(name='My Circuit from 2024-07-18 18:32:34.164662-QuantinuumBackend-final', description=None, properties=OrderedDict(), created=datetime.datetime(2024, 7, 18, 17, 32, 59, 616512, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 7, 18, 17, 32, 59, 616512, tzinfo=TzInfo(UTC))), project=ProjectRef(id=UUID('e1e53608-5e73-4b96-b15f-96e727f61e47'), annotations=Annotations(name='My Nexus Project', description=None, properties=OrderedDict(), created=datetime.datetime(2024, 7, 18, 17, 24, 2, 870970, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 7, 18, 17, 24, 2, 870970, tzinfo=TzInfo(UTC))), contents_modified=datetime.datetime(2024, 7, 18, 17, 36, 3, 844973, tzinfo=TzInfo(UTC)), type='ProjectRef'), type='CircuitRef')

In [16]:
# Get the results of the execution
result = execute_job_result_refs[0].download_result()

result.get_counts()

Counter({(0, 0): 100})

In [17]:
# Get the pytket BackendInfo to see the state of the device
execute_job_result_refs[0].download_backend_info()

BackendInfo(name='QuantinuumBackend snapshot', device_name='H1-1SC', version='0.36.0', architecture=<tket::FullyConnected, nodes=20>, gate_set={<OpType.Measure: 66>, <OpType.Reset: 68>, <OpType.Rz: 36>, <OpType.PhasedX: 71>, <OpType.Barrier: 8>, <OpType.ZZMax: 73>, <OpType.ClassicalExpBox: 107>, <OpType.ZZPhase: 76>, <OpType.TK2: 44>, <OpType.WASM: 14>, <OpType.SetBits: 15>, <OpType.CopyBits: 16>, <OpType.RangePredicate: 17>, <OpType.ExplicitPredicate: 18>, <OpType.ExplicitModifier: 19>, <OpType.MultiBit: 20>}, n_cl_reg=0, supports_fast_feedforward=True, supports_reset=True, supports_midcircuit_measurement=True, all_node_gate_errors=None, all_edge_gate_errors=None, all_readout_errors=None, averaged_node_gate_errors=None, averaged_edge_gate_errors=None, averaged_readout_errors=None, misc={'max_n_shots': 10000, 'options': {}, 'system_type': 'syntax checker', 'wasm': True})

### Managing Jobs

You can use the API to check on jobs, but also perform operations like cancelling or retrying.

In [18]:
# View your current jobs that are in the SUBMITTED state
qnx.jobs.get_all(job_status=[qnx.jobs.JobStatusEnum.SUBMITTED])

In [ ]:
other_execute_job_ref = qnx.start_execute_job(
    circuits=compiled_circuits,
    name=f"{my_job_name_prefix}_execute_other",
    n_shots=[100]* len(compiled_circuits),
    backend_config=qnx.QuantinuumConfig(device_name="H1-1SC"),
    project=my_project_ref,
)

In [ ]:
# Cancel the job
qnx.jobs.cancel(other_execute_job_ref)

In [ ]:
# Retry the job
qnx.jobs.retry_submission(
    other_execute_job_ref,
    retry_status=[qnx.jobs.StatusEnum.CANCELLED],
    remote_retry_strategy=qnx.jobs.RemoteRetryStrategy.FULL_RESTART)